In [1]:
import pandas as pd
import numpy as np

<p style='color: red;'>Important Note!<p> Do not forget to change the maximum capacity of each cluster.

In [2]:
raw = pd.read_csv('Data/raw_test.csv')

In [3]:
raw.head()

,ประทับเวลา,ชื่อ (Name),นามสกุล (Last Name),ชื่อเล่น (Nickname),รหัสนิสิต (Student ID),เบอร์โทรศัพท์ (Phone Number),E-mail,ภาควิชา (Department),1st Priority,2nd Priority,3rd Priority,4th Priority,5th Priority
0,2563/10/30 8:43:15 หลังเที่ยง GMT+7,Test,Test,Tes,Iikjbbn09888&,814025465,Hhk,Mining and Petroleum Engineering,Engineer : R&D Engineer,Business : Entrepreneur/Startups,Technology : Developer/Programmer,Business : Business Developer (BD) / Managemen...,Engineer : Production/Process/Plant Engineer
1,2563/10/30 9:02:21 หลังเที่ยง GMT+7,สาโรช,คุณวุฒิพร,ป๋วย,"53304796211,&:&",819212399,กวพรกน,ภาควิชาวิศวกรรมแหล่งน้ำ,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer


In [4]:
# Cluster List
clusters = [
    'Business : Business/Management Consultant',
    'Business : Business Developer (BD) / Management Trainee',
    'Business : Data Analyst',
    'Business : Entrepreneur/Startups',
    'Business : Investment Strategist',
    'Technology : AI Researcher/Data Engineer',
    'Technology : Developer/Programmer',
    'Technology : Project Manager/Product Owner',
    'Engineer : R&D Engineer',
    'Engineer : Production/Process/Plant Engineer'
]

In [5]:
# Create a capacity dict which defines maximum number of each
# cluster in each round
capacity_list = [2,3,2,4,2,5,5,3,8,2] # edit later
assert len(clusters) == len(capacity_list)
capacity = dict()
for c, max_no in zip(clusters, capacity_list):
    capacity[c] = max_no
    
#capacity

In [7]:
names = raw[['ชื่อ (Name)', 'นามสกุล (Last Name)', 'ชื่อเล่น (Nickname)',
             'รหัสนิสิต (Student ID)', 'เบอร์โทรศัพท์ (Phone Number)',
             'E-mail', 'ภาควิชา (Department)']]
#names.columns = ['name', 'surname', 'nickname', 'ID', 'tel',
#                 'email', 'department']
names.set_index('รหัสนิสิต (Student ID)', inplace=True)

ranks = raw[['รหัสนิสิต (Student ID)', '1st Priority', '2nd Priority',
             '3rd Priority', '4th Priority', '5th Priority']]
ranks.columns = ['ID', 'r1', 'r2', 'r3', 'r4', 'r5']
ranks.set_index('ID', inplace=True)

In [8]:
# Create a time-slot dict which states if an applicant is still
# vacant in a specific time-slot
# Note that True means vacant and False means not vacant
vacant_dict = dict()
for app in ranks.index:
    vacant_dict[app] = [True, True, True]
#vacant_dict

<h2>Algorithm:</h2><br>
We will create a dict with clusters as keys and three lists of applicants who are selected to be in such a cluster. Each list represents each round of the roundtable session.

In [9]:
rt_dict = dict()
for c in clusters:
    rt_dict[c] = [[], [], []]

In [10]:
for col in ranks.columns: # [r1, ... , r5]
    for app in ranks.index:
        # Check if an applicant is already assigned to three
        # clusters. If so, pass
        if vacant_dict[app] == [False, False, False]:
            pass
        
        cluster = ranks.loc[app, col]
        
        # If the first round is vacant, put this applicant
        # in the first round
        if len(rt_dict[cluster][0]) < capacity[cluster] and vacant_dict[app][0]:
            rt_dict[cluster][0].append(app)
            vacant_dict[app][0] = False
        
        # Else, if the second round is vacant, put this applicant
        # in the second round
        elif len(rt_dict[cluster][1]) < capacity[cluster] and vacant_dict[app][1]:
            rt_dict[cluster][1].append(app)
            vacant_dict[app][1] = False
            
        # Else, if the third round is vacant, put him in
        elif len(rt_dict[cluster][2]) < capacity[cluster] and vacant_dict[app][2]:
            rt_dict[cluster][2].append(app)
            vacant_dict[app][2] = False

In [11]:
rt_dict

{'Business : Business/Management Consultant': [[], [], []],
 'Business : Business Developer (BD) / Management Trainee': [[], [], []],
 'Business : Data Analyst': [[], [], []],
 'Business : Entrepreneur/Startups': [[], ['Iikjbbn09888&'], []],
 'Business : Investment Strategist': [[], [], []],
 'Technology : AI Researcher/Data Engineer': [['53304796211,&:&'],
  ['53304796211,&:&'],
  ['53304796211,&:&']],
 'Technology : Developer/Programmer': [[], [], ['Iikjbbn09888&']],
 'Technology : Project Manager/Product Owner': [[], [], []],
 'Engineer : R&D Engineer': [['Iikjbbn09888&'], [], []],
 'Engineer : Production/Process/Plant Engineer': [[], [], []]}

## Convert the result to a DataFrame

In [12]:
rounds = pd.DataFrame(index=ranks.index)

for cluster in rt_dict:
    for i in range(len(rt_dict[cluster])):
        column_to_insert = 'Round {}'.format(str(i + 1))
        rounds.loc[rt_dict[cluster][i], column_to_insert] = cluster

In [13]:
rounds

,Round 1,Round 2,Round 3
ID,,,
Iikjbbn09888&,Engineer : R&D Engineer,Business : Entrepreneur/Startups,Technology : Developer/Programmer
"53304796211,&:&",Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer


In [14]:
final_df = names.merge(rounds, how = 'inner', left_index = True,
                       right_index = True)
final_df

,ชื่อ (Name),นามสกุล (Last Name),ชื่อเล่น (Nickname),เบอร์โทรศัพท์ (Phone Number),E-mail,ภาควิชา (Department),Round 1,Round 2,Round 3
Iikjbbn09888&,Test,Test,Tes,814025465,Hhk,Mining and Petroleum Engineering,Engineer : R&D Engineer,Business : Entrepreneur/Startups,Technology : Developer/Programmer
"53304796211,&:&",สาโรช,คุณวุฒิพร,ป๋วย,819212399,กวพรกน,ภาควิชาวิศวกรรมแหล่งน้ำ,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer,Technology : AI Researcher/Data Engineer


# Checking

Check if everyone joins in their first preferred room.

In [17]:
first = ranks['r1']
first

ID
Iikjbbn09888&                       Engineer : R&D Engineer
53304796211,&:&    Technology : AI Researcher/Data Engineer
Name: r1, dtype: object

In [29]:
for app in first.index:
    assigned_clusters = rounds.loc[app, ['Round 1', 'Round 2', 'Round 3']].tolist()
    if first.loc[app] not in assigned_clusters:
        print(app)
        
# note that this should print nothing

In [30]:
# Save the result
#final_df.to_csv('Data/final.csv')